# Using the lmt-analysis library to export detection and event tables to Excel
---

### Import required packages

In [ ]:
import os
import sys
import numpy
import pandas
import sqlite3

sys.path.insert(1, "../")

from lmtanalysis import Measure
from lmtanalysis import Features
from lmtanalysis.Util     import getAllEvents
from lmtanalysis.Animal   import AnimalPool
from lmtanalysis.Event    import EventTimeLine
from lmtanalysis.FileUtil import getFilesToProcess
from scripts.Set_Genotype import set_genotype

In [ ]:
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')
get_ipython().magic('matplotlib qt5')

### Load an SQLite database

In [ ]:
files = getFilesToProcess()

if files and len(files) > 0:
    SQLITE_FN = files[0]
    out_dir  = os.path.dirname(SQLITE_FN)
    out_base = os.path.splitext(SQLITE_FN)[0]
else: print("No file selected...")

### Enter genotype (required for dyadic events table)

In [ ]:
set_genotype([SQLITE_FN])

### Connect to data base and load detections for first hour

In [ ]:
connection = sqlite3.connect( SQLITE_FN )

animalPool = AnimalPool()

# load infos about the animals
animalPool.loadAnimals( connection )

# load all detection (positions) of all animals for the first hour
animalPool.loadDetection( start = 0, end = Measure.oneHour)

# retrieve all event names
all_events = getAllEvents(connection=connection)

### Compute detection summary table for two different time ranges
 * 12 time bins: 0 to 60 minutes with 5 minutes long interval
 * 1 time bin  : 0 to 60 minutes with 60 minutes long interval

In [ ]:
detections_5min, detections_60min = Features.computeDetectionFeatures(animalPool, start="0min", end="60min", freq=["5min", "60min"])

# show
detections_60min

### Compute monadic event summary table for two different time ranges
 * 12 time bins: 0 to 60 minutes with 5 minutes long interval
 * 1 time bin  : 0 to 60 minutes with 60 minutes long interval

In [ ]:
events_5min, events_60min = Features.computeMonadicEventFeatures(animalPool, start="0min", end="60min", freq=["5min", "60min"])

# show one table
events_60min

### Compute dyadic event summary table grouped by genotype for two different time ranges

In [ ]:
dyadic_events = ["Approach", 
                 "Approach contact", 
                 "Approach rear", 
                 "Break contact", 
                 "Contact", 
                 "FollowZone Isolated", 
                 "Get away", 
                 "Group2", 
                 "Group3", 
                 "Group4", 
                 "Oral-genital Contact", 
                 "Oral-oral Contact", 
                 "Side by side Contact", 
                 "Side by side Contact, opposite way", 
                 "Social approach", 
                 "Train2", 
                 "seq oral geni - oral oral", 
                 "seq oral oral - oral genital"]

In [ ]:
events_dyadic_5min, events_dyadic_60min = Features.computeDyadicEventFeature(animalPool, dyadic_events, start="0min", end="60min", freq=["5min", "60min"])

# show
events_dyadic_60min

### Export to Excel file
stored in same folder as sqlite file, with 6 sheets:
 * Detection 5min
 * Detection 60min
 * Monadic Events 5min
 * Monadic Events 60min
 * Dyadic Events 5min
 * Dyadic Events 60min

In [ ]:
export = {}
export["Detection 5min"]  = detections_5min
export["Detection 60min"] = detections_60min

export["Monadic Events 5min"]     = events_5min
export["Monadic Events 60min"]    = events_60min

export["Dyadic Events 5min"]     = events_dyadic_5min
export["Dyadic Events 60min"]    = events_dyadic_60min

with pandas.ExcelWriter(out_base + ".xlsx") as excl_writer:
    for sheet_name, sheet_df in export.items():
        sheet_df.to_excel(excl_writer, sheet_name=sheet_name)